# Compare different parameter options

<div class="alert alert-block alert-info">
Scroll to the bottom.</div>

In [ ]:
import ipywidgets as widgets
from IPython.display import display
from PIL import Image
import os
import pandas as pd
from re import sub

In [ ]:
input_folder = os.path.dirname(os.path.realpath('__file__')) + "/figures"

In [ ]:
# dataframe with all files inside the folder
df = pd.DataFrame(os.listdir(input_folder), columns=["filename"])

# divide the filename into parts divided by "-" and create a new column for each part
df = df.join(df["filename"].str.split("-", expand=True).add_prefix("part_"))

# remove the file extension from the last column
df.iloc[:, -1] = df.iloc[:, -1].apply(lambda x: sub("\\..*", "", x))

# remove the filanem column
df = df.drop(columns=["filename"])

# replace column names
parameters = ["Count layer", "Normalization", "Scaling before PCA, UMAP, etc.", "Which genes to use for PCA, UMAP, etc.",
                "Method to remove cell cycle effects", "Number of PCs from PCA to use for UMAP", "Number of neighbors to use for UMAP",
                "Quality control method", "Embedding method", "Sample separately or combined", "Feature to color by"]
df.columns = parameters

df.head(2)

In [ ]:
import matplotlib.pyplot as plt

# Function to update images
def update_images(p0, p1, p2, p3, p4, p5, p6, p7, p8, p9, p10, compare_param):
    compare_param_index = parameters.index(compare_param)  # Get the index of the selected parameter
    compare_values = df[compare_param].unique()  # Get unique values of the selected parameter
    
    # Stop if the parameter to compare has only one value
    if len(compare_values) <= 1:
        print(f"The parameter '{compare_param}' has only one unique value. Please select a different parameter for comparison.")
        return
    
    images = []
    labels = []

    for value in compare_values:
        params = [p0, p1, p2, p3, p4, p5, p6, p7, p8, p9, p10]
        params[compare_param_index] = value  # Vary the selected parameter
        filename = f"{'-'.join(params)}.jpg"
        filepath = os.path.join(input_folder, filename)

        # Load image if it exists, else use a placeholder
        if os.path.exists(filepath):
            images.append(Image.open(filepath))
        else:
            images.append(Image.new('RGB', (100, 100), color='white'))
        
        labels.append(f"{df.columns[compare_param_index]}: {value}")  # Label for each image

    # Set up subplot layout
    num_images = len(images)
    if p8 == "combined":
        cols, rows = num_images, 1  # All images in one row
    else:
        cols, rows = 2, -(-num_images // 2)  # Two columns, adjust rows dynamically

    # Display images using matplotlib
    fig, axes = plt.subplots(rows, cols, figsize=(cols * 15, rows * 15))
    axes = axes.flatten() if num_images > 1 else [axes]  # Ensure iterable axes

    for i, (img, label) in enumerate(zip(images, labels)):
        axes[i].imshow(img)
        axes[i].axis("off")
        axes[i].set_title(label, fontsize=22)

    # Hide unused subplots if any
    for j in range(i + 1, len(axes)):
        axes[j].axis("off")

    plt.show()

# Create a layout with increased description width and dropdown width
dropdown_layout = widgets.Layout(width="600px")  # Increase width to prevent truncation
desc_style = {"description_width": "250px"}  # Adjust description width

dropdowns = [widgets.Dropdown(options=df[desc].unique(), description=desc, layout=dropdown_layout, style=desc_style) for i, desc in enumerate(parameters)]

# Dropdown for choosing parameter to compare
compare_dropdown = widgets.Dropdown(options=parameters, description="Compare different options of this parameter", layout=dropdown_layout, style=desc_style)

# Interactive function
widgets.interactive(update_images, 
                    p0=dropdowns[0], p1=dropdowns[1], p2=dropdowns[2], p3=dropdowns[3], 
                    p4=dropdowns[4], p5=dropdowns[5], p6=dropdowns[6], p7=dropdowns[7], 
                    p8=dropdowns[8], p9=dropdowns[9], p10=dropdowns[10],
                    compare_param=compare_dropdown)